In [ ]:
import cv2
import numpy as np
import os

def remove_horizontal_lines(image):
    # Convert the image to true black and white from grayscale
    threshold, image_bin = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    # Invert the image to change white to black and vice versa
    image_inv = 255 - image_bin

    # Define kernels for horizontal lines
    kernel_len = np.array(image).shape[1] // 100
    horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_len, 20))

    # Remove anything that is not a horizontal line
    image_inv = cv2.erode(image_inv, horizontal_kernel, iterations=3)
    horizontal_lines = cv2.dilate(image_inv, horizontal_kernel, iterations=5)

    # Subtract horizontal lines from the original image to remove them
    image_without_horizontal_lines = cv2.subtract(255 * np.ones_like(image), horizontal_lines)

    return image_without_horizontal_lines

# Path to the recorded video file
video_path = "C:\\Users\\User\\Downloads\\TheAware.AI\\mp4_converted_video_journal.mp4"

# Create an "output" folder if it doesn't exist
output_folder = "output"
os.makedirs(output_folder, exist_ok=True)

# Open the video file
cap = cv2.VideoCapture(0)

# Initialize variables to store the previous bounding box position
prev_x, prev_y, prev_w, prev_h = 50, 50, 50, 50
frame_count = 0

while True:
    # Read a frame from the video
    ret, frame = cap.read()

    if not ret:
        print("Finished")
        break

    # Apply Sobel operator for vertical gradient
    sobel_y = cv2.Sobel(frame, cv2.CV_64F, 0, 1, ksize=1)
    sobel_x = cv2.Sobel(frame, cv2.CV_64F, 1, 0, ksize=1)
    sobel_y = np.abs(sobel_y)
    sobel_x = np.abs(sobel_x)
    sobel_y = np.uint8(sobel_y)
    sobel_x = np.uint8(sobel_x)
    sobel = sobel_x + sobel_y

    # Apply Canny edge detection to the vertical gradient
    edges = cv2.Canny(sobel, 90, 150)

    # Remove horizontal lines
    new_image = remove_horizontal_lines(edges)
    # Define a rectangular kernel (you can adjust the size)
    #kernel = np.ones((5, 5), np.uint8)

    # Perform erosion
    #erosion_result = cv2.erode(new_image, kernel, iterations=1)
    # Find contours in the binary image
    contours, _ = cv2.findContours(new_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Sort contours based on their areas in descending order
    contours = sorted(contours, key=cv2.contourArea, reverse=True)

    # Draw a single rectangle around the largest contour that follows a certain area threshold
    if contours:
        max_contour = contours[0]
        if cv2.contourArea(max_contour) > 20000:
            x, y, w, h = cv2.boundingRect(max_contour)

            # Stabilize the bounding box by using the previous position
            x = int(0.9 * prev_x + 0.1 * x)
            y = int(0.9 * prev_y + 0.1 * y)
            w = int(0.9 * prev_w + 0.1 * w)
            h = int(0.9 * prev_h + 0.1 * h)
            # Save the processed frame with a unique filename
            frame_count += 1
            filename = os.path.join(output_folder, f"frame_{frame_count}.jpg")
            # cv2.imwrite(filename, frame)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            # Add text "Text Region" on the bounding box
            cv2.putText(frame, "Text Region", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)



            # Update the previous position
            prev_x, prev_y, prev_w, prev_h = x, y, w, h

    # Display the processed frame
    cv2.imshow("Processed Frame", frame)

    # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video and close all windows
cap.release()
cv2.destroyAllWindows()
